In [16]:
import numpy as np
gender = ['m','f','m','f','m']

f_count = 0
m_count = 0

for g in gender:
    if g == 'f':
        f_count += 1
    else:
        m_count += 1
print(f_count)
print(m_count)

2
3


In [17]:
## faster than for loop
f_count = len([el for el in gender if el == 'f'])
m_count = len([el for el in gender if el == 'm'])

print(f_count)
print(m_count)

2
3


In [18]:
## faster than for loop
f_count = sum([1 for el in gender if el == 'f'])
m_count = sum([1 for el in gender if el == 'm'])
print(f_count)
print(m_count)

2
3


In [12]:
## more Python 
from collections import Counter

Counter(gender)

Counter({'f': 2, 'm': 3})

In [25]:
import webget
import random


surnames_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.all.last'
female_names_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.female.first'
#male_names_txt = 'http://www2.census.gov/topics/genealogy/1990surnames/dist.male.first'

webget.download(surnames_txt, to='./last_names.txt')
webget.download(female_names_txt, to='./female_names.txt')
#webget.download(male_names_txt, to='./male_names.txt')


def get_names_from(file):
    names = []
    with open(file) as f:
        for line in f:
            name = line.split(' ')[0].title()
            names.append(name)
            
    return names

names = get_names_from('./last_names.txt')
female_names = get_names_from('./female_names.txt')
#male_names = get_names_from('./male_names.txt')

print(len(names), len(female_names))

88799 4275


In [28]:
def random_combination(list_a, list_b):
    a = random.choice(list_a)
    b = random.choice(list_b)
    
    return (a, b)


def generate_random_names(amount=1, gender='female'):
    pre_names = (female_names)
    
    if gender == 'female':
        pre_names = pre_names[0]
    elif gender == 'random':
        pre_names = random.choice(pre_names)

    random_names = []
    for idx in range(0, amount):    
        random_name = random_combination(pre_names, names)
        random_names.append(random_name)
        
    return random_names
    
generate_random_names(amount=10, gender='random')

[('r', 'Colon'),
 ('n', 'Whitsell'),
 ('a', 'Bohlsen'),
 ('s', 'Deturenne'),
 ('K', 'Mayhorn'),
 ('a', 'Kuning'),
 ('i', 'Tsing'),
 ('r', 'Westerfield'),
 ('a', 'Lowery'),
 ('K', 'Gherman')]

In [29]:
total_no_nodes = 100
no_nodes_per_sex = total_no_nodes // 2

female_names_list = generate_random_names(amount=no_nodes_per_sex, gender='female')
male_names_list = generate_random_names(amount=no_nodes_per_sex, gender='male')

all_names_list = female_names_list + male_names_list
person_list = all_names_list

print(len(all_names_list))
print(all_names_list[:20])

100
[('r', 'Misch'), ('M', 'Pecci'), ('r', 'Pistilli'), ('r', 'Rousu'), ('y', 'Thornbury'), ('M', 'Heape'), ('a', 'Hesketh'), ('M', 'Koebel'), ('y', 'Steedley'), ('y', 'Seltzen'), ('a', 'Willegal'), ('r', 'Lachut'), ('r', 'Schradle'), ('y', 'Schwenneker'), ('r', 'Morford'), ('M', 'Theus'), ('M', 'Daughton'), ('a', 'Hammerschmidt'), ('y', 'Roof'), ('M', 'Kuse')]


In [31]:
from tqdm import tqdm
from random import randint
from numpy.random import choice


def get_no_endorsments():
    relation_classes = {
        0: (0, 0),
        1: (1, 4),
        2: (5, 10),
        3: (11, 20),
        4: (21, 50),
        5: (51, 100)
    }
    p_distribution = [0.05, 0.36, 0.45, 0.1, 0.035, 0.005]

    no_relation_class = choice(list(relation_classes.keys()), 
                               p=p_distribution)
    no_endorsments = randint(relation_classes[no_relation_class][0], 
                             relation_classes[no_relation_class][1])
    return no_endorsments


def create_endorsements(node_list):
    """Creates a list of graph edges
    like [(1,10), (50,4), (99,20000), ...]"""
    endorsements = []
    idx_list = range(len(node_list))

    for idx, name_pair in tqdm(enumerate(node_list)):
        # get how many people this person (name_pair) endorses
        no_endorsments = get_no_endorsments()
        if no_endorsments > 0:
            # get a list of potential endorsements
            potential_endorsment_idxs = random.sample(idx_list, no_endorsments)
            if idx in potential_endorsment_idxs:
                # Removing myself in case I am in the list
                # cannot endorse myself
                idx_me_in_endorsements = potential_endorsment_idxs.index(idx)
                del potential_endorsment_idxs[idx_me_in_endorsements]
    
            endorsements += list(zip([idx for _ in range(len(potential_endorsment_idxs))], 
                                     potential_endorsment_idxs))
    return endorsements

endorsements = create_endorsements(person_list)

print(endorsements[:30])
print(person_list)

ModuleNotFoundError: No module named 'tqdm'

In [33]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [34]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.nx_agraph import graphviz_layout, write_dot


def create_graph():
    graph = nx.DiGraph()
    graph.clear()

    # add node by node, needed to add attributes...
    print(len(all_names_list))

    for idx, name_pair in enumerate(all_names_list):
        graph.add_node(idx, name=' '.join(name_pair))
    
    # graph.add_nodes_from(all_names_list)
    graph.add_edges_from(endorsements)

    return graph


def draw_graph(graph):
    nx.draw(graph, pos=graphviz_layout(graph), 
            node_size=30, width=.05, cmap=plt.cm.Blues, 
            with_labels=True, node_color=range(len(graph)))
    
graph = create_graph()
draw_graph(graph)

#nx.write_gml(graph, './social_network.gml')

100


NameError: name 'endorsements' is not defined

In [36]:
print(graph.node[10]['name'])
print(graph.out_degree(10))
print(graph.out_edges(10))
print(graph.node[graph.out_edges(10)[0][1]])
print(graph.in_edges(10))

NameError: name 'graph' is not defined

In [37]:
print(graph.node)
print(graph.number_of_nodes())

NameError: name 'graph' is not defined

In [39]:
import numpy as np


in_deg_vec = np.array([graph.in_degree(n) for n in graph.nodes()])
max_ind_deg = in_deg_vec.max()
print(np.argmax(in_deg_vec))
print(graph.node[np.argmax(in_deg_vec)]['name'])

NameError: name 'graph' is not defined

In [41]:

print(graph.node[27]['name'])

NameError: name 'graph' is not defined

In [42]:
## encodes the PageRank formula
# PR(p) = 1-d/N + d
# big sigma

# en list af node, with index nr
GLOBAL_PR = np.ones(graph.number_of_nodes())

# new list, som kigger på GLOBAL_PR list og hvad er på node list
# node 0 så kører dette funktion, repeat again and again!
def page_rank(node):
    damping = 0.85
    big_n = graph.number_of_nodes()
    pr_in = np.array([ GLOBAL_PR[remote] for remote, _ in graph.in_edges([node])])
    c = [graph.out_degree[remote] for remote, _ in graph.in_edges([node])]
    pr_p = ((1 - damping) / n) + damping * sum(pr_in /c)
    return pr_p

def compute_page_rank_step(graph):
    return np.array([page_rank(n) for n in grap.nodes()])

def compute_page_rank(graph,no_it = 100):
    global GLOBAL_PR
    for idx in range(no_it)

compute_page_rank_step(graph, no_it = 2)
GLOBAL_PR

SyntaxError: invalid syntax (<ipython-input-42-290cc84e263e>, line 23)

In [47]:
%%bash

curl https://api.github.com/search/repositories?q=language:python&sort=starts


{
  "total_count": 2441928,
  "incomplete_results": false,
  "items": [
    {
      "id": 21289110,
      "name": "awesome-python",
      "full_name": "vinta/awesome-python",
      "owner": {
        "login": "vinta",
        "id": 652070,
        "avatar_url": "https://avatars2.githubusercontent.com/u/652070?v=4",
        "gravatar_id": "",
        "url": "https://api.github.com/users/vinta",
        "html_url": "https://github.com/vinta",
        "followers_url": "https://api.github.com/users/vinta/followers",
        "following_url": "https://api.github.com/users/vinta/following{/other_user}",
        "gists_url": "https://api.github.com/users/vinta/gists{/gist_id}",
        "starred_url": "https://api.github.com/users/vinta/starred{/owner}{/repo}",
        "subscriptions_url": "https://api.github.com/users/vinta/subscriptions",
        "organizations_url": "https://api.github.com/users/vinta/orgs",
        "repos_url": "https://api.github.com/users/vinta/repos",
        "events_url

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  168k  100  168k    0     0   168k      0  0:00:01  0:00:01 --:--:--  123k


In [55]:
import requests

url = 'https://api.github.com/search/repositories?q=language:python&sort=starts'

r = requests.get(url)
print(type(r))
print(r.url)
print(r.status_code)
#print(r.json())

results = r.json()['items']
print(len(results))

<class 'requests.models.Response'>
https://api.github.com/search/repositories?q=language:python&sort=starts
200
30


In [60]:
import json
import api_keys
import requests

url = 'api.openweathermap.org/data/2.5/forecast'
query = {'q':'Copenhagen,dk',
         'mode': 'json',
         'appid': api_keys.OWM_API_KEY
}
r = requests.get(url,query)

ModuleNotFoundError: No module named 'api_keys'